In [6]:
import secretflow as sf
import spu
import ray

sf.shutdown()

sf.init(parties=['alice','bob','carol'], address='local')

2023-07-27 14:19:01,404	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [7]:
import numpy as np
from sklearn.datasets import load_iris

data, _ = load_iris(return_X_y=True,as_frame=True)
data['uid'] = np.arange(len(data)).astype('str')
data['month'] = ['July'] * (len(data) // 2) + ['August'] * (len(data) - len(data) // 2)

data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),uid,month
0,5.1,3.5,1.4,0.2,0,July
1,4.9,3.0,1.4,0.2,1,July
2,4.7,3.2,1.3,0.2,2,July
3,4.6,3.1,1.5,0.2,3,July
4,5.0,3.6,1.4,0.2,4,July
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,145,August
146,6.3,2.5,5.0,1.9,146,August
147,6.5,3.0,5.2,2.0,147,August
148,6.2,3.4,5.4,2.3,148,August


In [8]:
import os
import pandas

os.makedirs('.data', exist_ok=True)
data_alice = data.sample(frac=0.9)
data_bob = data.sample(frac=0.6)
data_carol = data.sample(frac=0.4)

data_alice.to_csv('.data/alice.csv',index=False)
data_bob.to_csv('.data/bob.csv',index=False)
data_carol.to_csv('.data/carol.csv',index=False)

In [9]:
import spu
# 创建一个集群的配置
cluster_def={
    'nodes': [
        {
            'party': 'alice',
            'address': '192.168.11.93:5000',
        },
        {
            'party': 'bob',
            'address': '192.168.11.93:5000',
        },
        {   'party': 'carol',
            'address': '192.168.11.93:5000',
        },
    ],
    'runtime_config': {
        'protocol': spu.spu_pb2.ABY3,
        'field': spu.spu_pb2.FM64,
        'sigmoid_mode': spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    }
}

alice = sf.PYU('alice')
bob = sf.PYU('bob')
carol = sf.PYU('carol')
# spu_psi = sf.SPU(cluster_def=cluster_def)
spu_psi = sf.SPU(sf.utils.testing.cluster_def(['alice','bob','carol']))
print(spu_psi)

In [13]:
input_path = {alice:'.data/alice.csv' , bob: '.data/bob.csv', carol: '.data/carol.csv'}
output_path = {alice:'.data/alice_psi.csv' , bob : '.data/bob_psi.csv', carol:'.data/carol_psi.csv'}
spu_psi.psi_csv(['uid', 'month'],input_path,output_path,'alice',protocol="ECDH_PSI_3PC")

[{'party': 'alice', 'original_count': 120, 'intersection_count': 30},
 {'party': 'bob', 'original_count': 105, 'intersection_count': 30},
 {'party': 'carol', 'original_count': 60, 'intersection_count': 30}]

(SPURuntime(device_id=None, party=alice) pid=8597) 2023-07-27 14:38:55.888 [info] [bucket_psi.cc:Init:234] bucket size set to 1048576
(SPURuntime(device_id=None, party=alice) pid=8597) 2023-07-27 14:38:55.889 [info] [bucket_psi.cc:CheckInput:149] Begin sanity check for input file: .data/alice.csv, precheck_switch:true
(SPURuntime(device_id=None, party=alice) pid=8597) 2023-07-27 14:38:55.890 [info] [csv_checker.cc:CsvChecker:121] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=.data --stable selected-keys.1690439935890012000 | LC_ALL=C uniq -d > duplicate-keys.1690439935890012000
(SPURuntime(device_id=None, party=alice) pid=8597) 2023-07-27 14:38:55.897 [info] [bucket_psi.cc:CheckInput:166] End sanity check for input file: .data/alice.csv, size=120
(SPURuntime(device_id=None, party=alice) pid=8597) 2023-07-27 14:38:55.897 [info] [bucket_psi.cc:RunPsi:267] Run psi protocol=4, self_items_count=120
(SPURuntime(device_id=None, party=alice) pid=8597) 2023-